In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties

from utils.model_score import *
import seaborn as sns
from matplotlib.font_manager import FontProperties
import pickle

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)
    
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

[Errno 2] No such file or directory: 'C:\\System\\Library\\Fonts\\PingFang.ttc'


In [2]:
prefix = "./ModelPerformanceSeed"
models = ["logistic", "svc", "xgboost"]
variable_names = [
    'pass_out_overlap', 'pass_0', 'pass_1',
    'car_out_overlap', 'car_0', 'car_1', 'car_2',
    'motor_out_overlap', 'motor_0', 'motor_1',
    'full_data'
]
file_paths = []
seeds = [40] * 11 + [41] * 11 + [42] * 11 + [43] * 11 + [44] * 11 + [45] * 11 + [46] * 11 + [47] * 11 + [48] * 11 + [49] * 11

for model in models:
    for seed in range(40, 50):
        for var_name in variable_names:
            file_paths.append(f"{prefix}/{seed}/{model}/{var_name}.pkl")

def load_pickle_files(file_paths, variable_names):
    data = {}
    # 10個seed需要將variable_names重複10次
    for file_path, var_name, seed in zip(file_paths, variable_names * 10, seeds):
        if seed not in data:
            data[seed] = {}  # 只在 seed 不存在時初始化
        with open(file_path, "rb") as f:
            data[seed][var_name] = pickle.load(f)
    return data

xgb_data = load_pickle_files(file_paths[:110], variable_names)
svc_data = load_pickle_files(file_paths[110:220], variable_names)
logistic_data = load_pickle_files(file_paths[220:], variable_names)

In [3]:
logistic_final_data = {}
svc_final_data = {}
xgb_final_data = {}

for model in models:
    for seed in range(40, 50):
        for variable_name in variable_names:
            if model == "logistic":
                if seed not in logistic_final_data:
                    logistic_final_data[seed] = {}  # 只在 seed 不存在時初始化
                logistic_final_data[seed][variable_name] = get_score(
                    logistic_data[seed][variable_name]['y'], 
                    logistic_data[seed][variable_name]['decision_scores']
                )

            elif model == "svc":
                if seed not in svc_final_data:
                    svc_final_data[seed] = {}
                svc_final_data[seed][variable_name] = get_score(
                    svc_data[seed][variable_name]['y'], 
                    svc_data[seed][variable_name]['decision_scores']
                )

            elif model == "xgboost":
                if seed not in xgb_final_data:
                    xgb_final_data[seed] = {}
                xgb_final_data[seed][variable_name] = get_score(
                    xgb_data[seed][variable_name]['y'], 
                    xgb_data[seed][variable_name]['decision_scores']
                )


In [4]:
tp_mtrx_log = {}
fd_mtrx_log = {}
tp_mtrx_svc = {}
fd_mtrx_svc = {}
tp_mtrx_xgb = {}
fd_mtrx_xgb = {}

for seed in range(40, 50):
    tp_mtrx_log[seed] = np.zeros((2, 2))
    fd_mtrx_log[seed] = np.zeros((2, 2))
    tp_mtrx_svc[seed] = np.zeros((2, 2))
    fd_mtrx_svc[seed] = np.zeros((2, 2))
    tp_mtrx_xgb[seed] = np.zeros((2, 2))
    fd_mtrx_xgb[seed] = np.zeros((2, 2))
    for dataset, values in logistic_final_data[seed].items():
        if dataset != "full_data":
            tp_mtrx_log[seed] += values[0]
        elif dataset == "full_data":
            fd_mtrx_log[seed] += values[0]
    for dataset, values in svc_final_data[seed].items():
        if dataset != "full_data":
            tp_mtrx_svc[seed] += values[0]
        elif dataset == "full_data":
            fd_mtrx_svc[seed] += values[0]
    for dataset, values in xgb_final_data[seed].items():
        if dataset != "full_data":
            tp_mtrx_xgb[seed] += values[0]
        elif dataset == "full_data":
            fd_mtrx_xgb[seed] += values[0]

In [13]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(data_dict):
    precision_list = []
    recall_list = []
    f1_list = []
    accuracy_list = []
    total_sum = 0

    for seed, matrix in data_dict.items():
        FP = matrix[0, 1]
        TN = matrix[0, 0]
        FN = matrix[1, 0]
        TP = matrix[1, 1]
        
        
        total_sum += matrix.sum()
        
        y_true = np.array([1] * int(TP) + [1] * int(FN) + [0] * int(FP) + [0] * int(TN))
        y_pred = np.array([1] * int(TP) + [0] * int(FN) + [1] * int(FP) + [0] * int(TN))


        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
        accuracy = accuracy_score(y_true, y_pred)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        accuracy_list.append(accuracy)

    return {
        "Recall_mean": np.mean(recall_list), "Recall_std": np.std(recall_list, ddof=1),
        "Precision_mean": np.mean(precision_list), "Precision_std": np.std(precision_list, ddof=1),
        "F1-Score_mean": np.mean(f1_list), "F1-Score_std": np.std(f1_list, ddof=1),
        "Accuracy_mean": np.mean(accuracy_list), "Accuracy_std": np.std(accuracy_list, ddof=1),
        "Avg": total_sum
    }

def round_metrics(metrics_dict):
    return {key: round(value*100, 1) for key, value in metrics_dict.items()}


In [14]:
# Define model names and corresponding confusion matrices
models = {
    "Topology Log": (tp_mtrx_log, fd_mtrx_log),
    "Topology SVC": (tp_mtrx_svc, fd_mtrx_svc),
    "Topology XGB": (tp_mtrx_xgb, fd_mtrx_xgb)
}

# Initialize lists to store results
data_rows = []

# Compute metrics in a loop
for model_name, (tp_mtrx, fd_mtrx) in models.items():
    tp_metrics = round_metrics(compute_metrics(tp_mtrx))
    fd_metrics = round_metrics(compute_metrics(fd_mtrx))

    data_rows.append([model_name, tp_metrics["Recall_mean"], tp_metrics["Recall_std"],
                      tp_metrics["Precision_mean"], tp_metrics["Precision_std"],
                      tp_metrics["F1-Score_mean"], tp_metrics["F1-Score_std"],
                      tp_metrics["Accuracy_mean"], tp_metrics["Accuracy_std"]])

    data_rows.append([f"Full {model_name.split()[1]}", fd_metrics["Recall_mean"], fd_metrics["Recall_std"],
                      fd_metrics["Precision_mean"], fd_metrics["Precision_std"],
                      fd_metrics["F1-Score_mean"], fd_metrics["F1-Score_std"],
                      fd_metrics["Accuracy_mean"], fd_metrics["Accuracy_std"]])

# Create DataFrame
df = pd.DataFrame(data_rows, columns=[
    "Dataset", "Recall_mean", "Recall_std", "Precision_mean", "Precision_std",
    "F1-Score_mean", "F1-Score_std", "Accuracy_mean", "Accuracy_std"
])

df

,Dataset,Recall_mean,Recall_std,Precision_mean,Precision_std,F1-Score_mean,F1-Score_std,Accuracy_mean,Accuracy_std
0,Topology Log,64.3,5.6,71.4,2.8,67.5,2.3,69.1,1.2
1,Full Log,61.4,14.7,59.4,3.0,59.2,6.6,59.1,1.2
2,Topology SVC,64.6,3.7,75.3,3.4,69.4,1.1,71.5,1.0
3,Full SVC,55.0,9.9,72.9,3.4,62.0,5.2,66.9,1.9
4,Topology XGB,63.0,5.7,76.8,3.8,68.9,2.3,71.7,1.0
5,Full XGB,56.4,9.6,71.7,2.4,62.6,5.0,66.9,1.9


In [ ]:
metrics = ["Recall", "Precision", "F1-Score", "Accuracy"]
colors = ["blue", "green", "red", "purple"]
x = np.arange(len(df["Dataset"]))
# Create separate scatter plots for each metric without overlaying
fig, axes = plt.subplots(2, 2, figsize=(12, 10))  # Create a 2x2 grid of subplots

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Plot each metric separately
for i, metric in enumerate(metrics):
    mean_values = df[f"{metric}_mean"]
    std_values = df[f"{metric}_std"]
    
    axes[i].errorbar(x, mean_values, yerr=std_values, fmt='o', capsize=5, color=colors[i])
    
    # X-axis labels
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(df["Dataset"])
    
    # Labels and title
    axes[i].set_xlabel("Dataset")
    axes[i].set_ylabel("Score (%)")
    axes[i].set_title(f"{metric}")
    
    for j, (mean, std) in enumerate(zip(mean_values, std_values)):
        axes[i].text(j, mean, f"{mean:.1f} ± {std:.1f}", fontsize=9, va='bottom')

    # Grid
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()
plt.show()

# 細分群體

In [44]:
results = {}

# 針對每個變數計算
for var in variable_names:
    data_dict = {seed: svc_final_data[seed][var][0] for seed in range(40, 50)}  # 提取混淆矩陣
    results[var] = round_metrics(compute_metrics(data_dict))
    
metrics_df = pd.DataFrame.from_dict(results, orient="index")

metrics_df['Avg'] = metrics_df['Avg'].apply(lambda x: x/1000)
metrics_df

,Recall_mean,Recall_std,Precision_mean,Precision_std,F1-Score_mean,F1-Score_std,Accuracy_mean,Accuracy_std,Avg
pass_out_overlap,80.8,13.6,92.3,13.5,84.9,8.8,85.5,9.5,9.0
pass_0,85.7,30.7,83.8,18.8,78.7,22.6,80.0,16.6,7.6
pass_1,79.2,12.5,77.2,12.3,77.1,8.5,76.6,8.9,21.8
car_out_overlap,51.3,13.3,81.5,11.7,60.9,8.5,68.4,2.7,48.0
car_0,65.9,11.0,70.5,7.6,67.1,3.7,68.1,3.0,165.6
car_1,64.6,10.4,78.2,5.5,70.2,6.1,73.0,4.0,100.0
car_2,55.5,24.6,84.6,15.8,61.8,19.2,70.0,9.3,21.0
motor_out_overlap,61.7,24.0,93.0,12.0,70.7,18.5,77.3,11.5,9.2
motor_0,67.2,6.2,79.7,5.8,72.6,3.4,74.7,3.2,187.6
motor_1,56.9,18.5,74.2,10.7,62.0,10.0,67.1,4.9,52.4


In [ ]:
metrics = ["Recall", "Precision", "F1-Score", "Accuracy"]
colors = ["blue", "green", "red", "purple"]
x = np.arange(len(metrics_df.index))
# Create separate scatter plots for each metric without overlaying
fig, axes = plt.subplots(2, 2, figsize=(15, 10))  # Create a 2x2 grid of subplots

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Plot each metric separately
for i, metric in enumerate(metrics):
    mean_values = metrics_df[f"{metric}_mean"]
    std_values = metrics_df[f"{metric}_std"]
    
    # Ensure upper bound does not exceed 100
    upper_bounds = np.minimum(mean_values + std_values, 100)

    # Error bar plot with capped standard deviation
    axes[i].errorbar(x, mean_values, yerr=[std_values, upper_bounds - mean_values], fmt='o', capsize=5, color=colors[i])
    
    # X-axis labels
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(metrics_df.index, rotation=40)
    
    # Labels and title
    axes[i].set_xlabel("Dataset")
    axes[i].set_ylabel("Score (%)")
    axes[i].set_title(f"{metric}")
    
    for j, (mean, std, upper) in enumerate(zip(mean_values, std_values, upper_bounds)):
        display_std = upper - mean  # Adjusted std to prevent exceeding 100
        axes[i].text(j, mean, f"{mean:.1f} ± {display_std:.1f}", fontsize=7, va='bottom')

    # Grid
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout
plt.tight_layout()
plt.show()